In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import time
import heapq

G = nx.karate_club_graph()


def run_ic(G, seeds, p=0.1, simulations=100):
    total_spread = 0
    for _ in range(simulations):
        active = set(seeds)
        new_active = set(seeds)
        while new_active:
            next_active = set()
            for node in new_active:
                for neighbor in G.neighbors(node):
                    if neighbor not in active and random.random() < p:
                        next_active.add(neighbor)
            new_active = next_active
            active |= next_active
        total_spread += len(active)
    return total_spread / simulations


def greedy(G, k=3, p=0.1, simulations=100):
    S = []
    for _ in range(k):
        best_node = None
        best_gain = -1
        for node in G.nodes():
            if node in S:
                continue
            gain = run_ic(G, S + [node], p, simulations)
            if gain > best_gain:
                best_gain = gain
                best_node = node
        S.append(best_node)
    return S


def celf(G, k=3, p=0.1, simulations=100):
    S = []
    marg_gain = []
    node_gain = {}

    for node in G.nodes():
        gain = run_ic(G, [node], p, simulations)
        marg_gain.append((-gain, node, 0))
        node_gain[node] = gain

    heapq.heapify(marg_gain)
    cur_iter = 1
    lookups = 0

    while len(S) < k:
        gain, node, prev_iter = heapq.heappop(marg_gain)
        gain = -gain
        if prev_iter == len(S):
            S.append(node)
        else:
            new_gain = run_ic(G, S + [node], p, simulations) - run_ic(G, S, p, simulations)
            heapq.heappush(marg_gain, (-new_gain, node, len(S)))
            lookups += 1

    return S, lookups


k = 3
p = 0.1
simulations = 100

start = time.time()
greedy_seeds = greedy(G, k, p, simulations)
greedy_time = time.time() - start

start = time.time()
celf_seeds, lookups = celf(G, k, p, simulations)
celf_time = time.time() - start

print(f"Greedy Seeds: {greedy_seeds}, Time: {greedy_time:.4f} seconds")
print(f"CELF Seeds:   {celf_seeds}, Time: {celf_time:.4f} seconds")
print(f"CELF was {greedy_time / celf_time:.2f}x faster with {lookups} lazy evaluations.")


Greedy Seeds: [0, 33, 32], Time: 0.0542 seconds
CELF Seeds:   [33, 0, 32], Time: 0.0266 seconds
CELF was 2.04x faster with 11 lazy evaluations.
